# 层结构以及初始化方法
## 1、卷积层
<img src="/mnt/mydisk2/myPytorch/pic/卷积层.png" alt="FAO" width="790">

In [2]:
import torch.nn as nn

conv = nn.Conv2d(3,64,3,1,0)
print(conv)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))


### 计算输出特征图的大小
- 假设当前图像的大小为：Hin*Win
- 假设卷积核大小为: FH * FW ;个数为FN
- 填充数(padding)为P，步长 (stride)为S
- 输出图像的大小为: Hout * Wout
- 输出图像维度为:(FN, Hout, Wout)
- 其中Padding如果取VALID模式，则p=0;如果取SAME,则p>0

i) 不含膨胀率的计算方法:  
Hout = (Hin + 2*P - FH) / S + 1  
Wout = (Win + 2*P - FH) / S + 1  
ii) 含膨胀率的计算方法:  
Hout = (Hin + 2*P - D*(FH-1) - 1) / S + 1  
Wout = (Win + 2*P - D*(FH-1) - 1) / S + 1  

## 反卷积
#### 反卷积就是卷积的反向操作
<img src="/mnt/mydisk2/myPytorch/pic/反卷积演示.png" alt="FAO" width="790">

反卷积的两种方法：  
1、双线性插值上采样  
<img src="/mnt/mydisk2/myPytorch/pic/反卷积_双线性插值上采样.png" alt="FAO" width="790">

`bilinear_layer = nn.UpsamplingBilinear2d(size=None,scale_factor=None)`  
size表示期望的输出尺寸，scale_factor表示缩放因子，用来决定缩放的大小

In [6]:
import torch
import torch.nn as nn

# 定义一个卷积层，用来提取图像特征
conv = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
# 假设输入的图像大小是 28x28，经过卷积层后，得到一个 14x14 的特征图
x = torch.randn(1, 1, 28, 28)
print(x.shape)
x = conv(x)
print(x.shape)

# 定义一个上采样模块，用来恢复特征图大小
up_bilinear = nn.UpsamplingBilinear2d(size=(56,56))
x = up_bilinear(x)
print(x.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 14, 14])
torch.Size([1, 16, 56, 56])


2、转置卷积  
（1）转置卷积通过学习的方式，即通过在训练中更新卷积核的参数，以完成上采样过程  
（2）其计算结果往往更具鲁棒性  
（3）缺点是会增加模型的训练时间和训练参数  
（4）其具体代码与卷积代码类似：比卷积层仅多了一个输出填充参数，其他参数均不变  
`transpose_conv = nn.ConvTranspose2d(in_channels,out_channels,kernel_size,stride=1,padding=0,output_padding=0,group=1,bias=True)`

# 2、池化层
<img src="/mnt/mydisk2/myPytorch/pic/池化层.png" alt="FAO" width="790">  
 
1、最大池化

`
maxpool_layer = nn.MaxPool2d(kernel_size,stride=None,padding=0,dilation=1,return_indices=False,ceil_mode=False)
`

2、平均池化  

`average_layer = nn.AvgPool2d(kernel_size,stride=None,padding=0,ceil_mode=False,count_include_pad=True)`

- kernel_size(int or tuple) - max pooling的窗口大小  
- stride(int or tuple, optional) - max pooling的窗口移动的步长。默认值是- kernel_size
- padding(int or tuple, optional) - 输入的每一条边补充0的层数
- dilation(int or tuple, optional) – 一个控制窗口中元素步幅的参数
- return_indices - 如果等于True，会返回输出最大值的序号，对于上采样操作会有帮助
- ceil_mode - 如果等于True，计算输出信号大小的时候，会使用向上取整，代替默认的向下取整的操作

3、Mixed Pooling（对固定区域随机最大和平均池化）  

4、Stochastic Pooling（对随机区域随机最大和平均池化）  

<u>采用平均池化和最大池化最终一般会得到类似的结果</u>

# 3、正则化层
<img src="/mnt/mydisk2/myPytorch/pic/正则化层.png" alt="FAO" width="790"> 

&emsp;&emsp;正则化层是神经网络中非常重要的操作之一其全称为Batch它通过将数据进Normalization (BN)，也就是标准化外理。行偏移和尺度缩放调整，在数据预处理时是非常常见的操作，在网络的中间层如今也很频繁的被使用。
- 完成对数据的便宜和缩放，使其保持原有的分布特征，从而补偿网络的非线性表达能力损失
- 好处：  
（1）减轻对初始数据的依赖  
（2）加速训练，学习率可以设置的更高
- 坏处：  
依赖batch的大小，batch不同，方差和均值的计算不稳定，导致BN不适合batch较小的场景，不适合RNN

`BN = nn.BatchNorm2d(channel)`

# 4、激活函数
<img src="/mnt/mydisk2/myPytorch/pic/激活函数.png" alt="FAO" width="790"> 

- 在实际应用中，我们用神经网络期望解决的问题往往是非线性的，这就需要我们引入更多的非线性因素，而激活函数便承担了此功能。

`relu = nn.ReLU()`

# 5、全连接层
<img src="/mnt/mydisk2/myPytorch/pic/全连接层.png" alt="FAO" width="790"> 

全连接层是卷积神经网络中重要的组成部分之一，起到“分类器”的作用全连接层的作用，是将特征图的分布式表示映射到样本标记空间。通俗来讲，就是将某一特征图的表示方式整合在一起，最终输出为一个数值